# Signate開始時のお約束

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [3]:
!pip install signate

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=14e1bd3d2f2f2f051aac868fe1825a518f183096ca5a49d4ef9887ab0315277f
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [4]:
!signate files --competition-id=1523

  fileId  name               title                     size  updated_at
--------  -----------------  ----------------------  ------  -------------------
    3939  train.csv          学習用データ            494008  2024-08-01 12:00:28
    3940  test.csv           評価用データ            488363  2024-08-01 12:00:38
    3944  sample_submit.csv  応募用サンプルファイル   86893  2024-08-01 12:00:49
    4057  tutorial.ipynb     チュートリアルコード    323126  2024-08-01 12:01:00


In [5]:
!signate download --competition-id=1523

sample_submit.csv

tutorial.ipynb

test.csv

train.csv


Download completed.


In [6]:
!pip install kanjize

In [7]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 12.1 MB/s eta 0:00:00


In [8]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 26.4 MB/s eta 0:00:00


In [9]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# データフレーム読み込み
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

# データ結合(番号を振り直す)
all_df = pd.concat([train_df,test_df],axis=0,ignore_index=True)
max_row = len(all_df)

# データ処理開始

# Ageの処理

In [10]:
###############
# "Age"の処理 #
###############
import unicodedata # 全角を半角に変換
from kanjize import kanji2number # 漢字を数値に変換
import re

# 漢字の判定用
p = re.compile('[一二三四五六七八九十]+')

# nanを-1歳で補完
all_df["Age"] = all_df["Age"].fillna("-1")

# Ageの処理
for i in range(len(all_df)):
# for i in range(max_row):

  # 末尾文字の取得
  tail = all_df.loc[i,"Age"][-1:]

  # 先頭文字が漢字の場合
  if(p.search(all_df.loc[i,"Age"][0])!=None):
    tmp = all_df.loc[i,"Age"][:-1]
    all_df.loc[i,"Age"] = kanji2number(tmp)
    all_df.loc[i,"Age"] = int(all_df.loc[i,"Age"])

  # 末尾文字が歳、才、際の場合
  elif((tail=="歳")or(tail=="才")or(tail=="際")):
    all_df.loc[i,"Age"] = all_df.loc[i,"Age"][:-1]
    # 全角を半角に直す
    all_df.loc[i,"Age"] = unicodedata.normalize("NFKC", all_df.loc[i,"Age"])
    all_df.loc[i,"Age"] = int(all_df.loc[i,"Age"])

  # 代の場合
  elif(tail == "代"):
    tmp = all_df.loc[i,"Age"][:-1]

    # 「代」は○5歳に変換する
    tmp = int(tmp)
    # 10代は19歳にする
    if(-1<tmp & tmp<20):
      all_df.loc[i,"Age"] = 19
    elif(tmp>60):
      all_df.loc[i,"Age"] = 61
    else:
      # 10歳で分けて5足す
      tmp = tmp//10 * 10
      all_df.loc[i,"Age"] = tmp + 5

# int型に変換
all_df["Age"] = all_df["Age"].astype(int)

print(all_df.groupby("Age").size())

Age
-1     193
 18     17
 19     28
 20     47
 21     58
 22     81
 23     74
 24     96
 25    193
 26    123
 27    122
 28    160
 29    158
 30    194
 31    217
 32    234
 33    263
 34    234
 35    469
 36    257
 37    238
 38    206
 39    204
 40    188
 41    186
 42    198
 43    170
 44    129
 45    354
 46    146
 47    144
 48    125
 49    136
 50    165
 51    156
 52    154
 53    131
 54    131
 55    252
 56     97
 57     86
 58     63
 59     53
 60     30
 61     14
 65      4
dtype: int64


In [11]:
# カテゴリ列の追加
all_df["Age_cls"] = all_df["Age"].copy()

all_df.loc[all_df["Age"]<0,"Age_cls"] = "不明"
all_df.loc[(all_df["Age"]>=10) & (all_df["Age"]<=20),"Age_cls"] = "20歳以下"
all_df.loc[(all_df["Age"]>=21) & (all_df["Age"]<=25),"Age_cls"] = "21〜25歳"
all_df.loc[(all_df["Age"]>=26) & (all_df["Age"]<=30),"Age_cls"] = "26〜30歳"
all_df.loc[(all_df["Age"]>=31) & (all_df["Age"]<=35),"Age_cls"] = "31〜35歳"
all_df.loc[(all_df["Age"]>=36) & (all_df["Age"]<=40),"Age_cls"] = "36〜40歳"
all_df.loc[(all_df["Age"]>=41) & (all_df["Age"]<=45),"Age_cls"] = "41〜45歳"
all_df.loc[(all_df["Age"]>=46) & (all_df["Age"]<=50),"Age_cls"] = "46〜50歳"
all_df.loc[(all_df["Age"]>=51) & (all_df["Age"]<=55),"Age_cls"] = "51〜55歳"
all_df.loc[all_df["Age"]>=56,"Age_cls"] = "56歳以上"

# 元のカラムを削除する
# all_df = all_df.drop("Age",axis=1)

all_df['Age_cls'] = all_df['Age_cls'].astype('object')
print(all_df.groupby(["Age_cls"]).size())

Age_cls
20歳以下       92
21〜25歳     502
26〜30歳     757
31〜35歳    1417
36〜40歳    1093
41〜45歳    1037
46〜50歳     716
51〜55歳     824
56歳以上      347
不明         193
dtype: int64


<ipython-input-11-6e1667c3d861>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '不明' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_df.loc[all_df["Age"]<0,"Age_cls"] = "不明"


In [12]:
print(all_df[:len(train_df)].groupby("Age").size())

Age
-1     100
 18      6
 19     12
 20     28
 21     34
 22     43
 23     38
 24     38
 25     99
 26     67
 27     56
 28     74
 29     74
 30     87
 31    109
 32    108
 33    127
 34    112
 35    232
 36    134
 37    118
 38    107
 39    105
 40     95
 41     94
 42     96
 43     91
 44     62
 45    193
 46     75
 47     69
 48     68
 49     68
 50     76
 51     83
 52     83
 53     71
 54     61
 55    128
 56     49
 57     36
 58     33
 59     25
 60     16
 61      5
 65      4
dtype: int64


In [13]:
#########################
# "TypeofContact"の処理 #
#########################
# all_df["TypeofContact"] = all_df["TypeofContact"].fillna("Self Enquiry")
print(all_df.groupby("TypeofContact").size())

TypeofContact
Company Invited    2516
Self Enquiry       4444
dtype: int64


In [14]:
print(all_df[:len(train_df)].groupby("TypeofContact").size())

TypeofContact
Company Invited    1289
Self Enquiry       2194
dtype: int64


In [15]:
#########################
# "CityTier"の処理 #
#########################
print(all_df.groupby("CityTier").size())

CityTier
1    2991
2    3160
3     827
dtype: int64


In [16]:
print(all_df[:len(train_df)].groupby("CityTier").size())

CityTier
1    1475
2    1596
3     418
dtype: int64


In [17]:
#########################
# "DurationOfPitch"の処理 #
#########################
# nanを-1分で補完
all_df["DurationOfPitch"] = all_df["DurationOfPitch"].fillna("-1分")

# 秒を分に変換 #
for i in range(len(all_df)):

  tail = str(all_df.loc[i,"DurationOfPitch"])[-1:]

  if(tail == "秒"):
    tmp = all_df.loc[i,"DurationOfPitch"][:-1]
    all_df.loc[i,"DurationOfPitch"] = float(tmp) / 60.0
  elif(tail == "分"):
    all_df.loc[i,"DurationOfPitch"] = all_df.loc[i,"DurationOfPitch"][:-1]

# int型に変換
all_df["DurationOfPitch"] = all_df["DurationOfPitch"].astype("int")

# 秒に換算
all_df["DurationOfPitch"] = all_df["DurationOfPitch"] * 60
print(all_df.groupby("DurationOfPitch").size())

DurationOfPitch
-60      252
 240       3
 300      43
 360     167
 420     446
 480     671
 540     654
 600     467
 660     306
 720     293
 780     397
 840     518
 900     569
 960     527
 1020    312
 1080    133
 1140     50
 1200     66
 1260     65
 1320     84
 1380    104
 1440     86
 1500     79
 1560     64
 1620     57
 1680     56
 1740     48
 1800     65
 1860     86
 1920     93
 1980     69
 2040     75
 2100     58
 2160     15
dtype: int64


In [18]:
# カテゴリ列の追加
all_df["DurationOfPitch_cls"] = all_df["DurationOfPitch"].copy()

all_df.loc[all_df["DurationOfPitch"]<0,"DurationOfPitch_cls"] = "不明"
all_df.loc[(all_df["DurationOfPitch"]>0) & (all_df["DurationOfPitch"]<=400),"DurationOfPitch_cls"] = "400s以下"
all_df.loc[(all_df["DurationOfPitch"]>400) & (all_df["DurationOfPitch"]<=500),"DurationOfPitch_cls"] = "401〜500s"
all_df.loc[(all_df["DurationOfPitch"]>500) & (all_df["DurationOfPitch"]<=600),"DurationOfPitch_cls"] = "501〜600s"
all_df.loc[(all_df["DurationOfPitch"]>600) & (all_df["DurationOfPitch"]<=700),"DurationOfPitch_cls"] = "601〜700s"
all_df.loc[(all_df["DurationOfPitch"]>700) & (all_df["DurationOfPitch"]<=800),"DurationOfPitch_cls"] = "701〜800s"
all_df.loc[(all_df["DurationOfPitch"]>800) & (all_df["DurationOfPitch"]<=900),"DurationOfPitch_cls"] = "801〜900s"
all_df.loc[(all_df["DurationOfPitch"]>900) & (all_df["DurationOfPitch"]<=1000),"DurationOfPitch_cls"] = "901〜1000s"
all_df.loc[(all_df["DurationOfPitch"]>1000) & (all_df["DurationOfPitch"]<=1100),"DurationOfPitch_cls"] = "1001〜1100s"
all_df.loc[(all_df["DurationOfPitch"]>1100) & (all_df["DurationOfPitch"]<=1200),"DurationOfPitch_cls"] = "1101〜1200s"
all_df.loc[(all_df["DurationOfPitch"]>1200) & (all_df["DurationOfPitch"]<=1300),"DurationOfPitch_cls"] = "1201〜1300s"
all_df.loc[(all_df["DurationOfPitch"]>1300) & (all_df["DurationOfPitch"]<=1400),"DurationOfPitch_cls"] = "1301〜1400s"
all_df.loc[(all_df["DurationOfPitch"]>1400) & (all_df["DurationOfPitch"]<=1500),"DurationOfPitch_cls"] = "1401〜1500s"
all_df.loc[all_df["DurationOfPitch"]>1500,"DurationOfPitch_cls"] = "1501s以上"

# 元のカラムを削除
# all_df = all_df.drop("DurationOfPitch", axis=1)

all_df["DurationOfPitch_cls"] = all_df["DurationOfPitch_cls"].astype("object")
print(all_df.groupby(["DurationOfPitch_cls"]).size())

DurationOfPitch_cls
1001〜1100s     445
1101〜1200s     116
1201〜1300s      65
1301〜1400s     188
1401〜1500s     165
1501s以上        686
400s以下         213
401〜500s      1117
501〜600s      1121
601〜700s       306
701〜800s       690
801〜900s      1087
901〜1000s      527
不明             252
dtype: int64


<ipython-input-18-57a532ae39e6>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '不明' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_df.loc[all_df["DurationOfPitch"]<0,"DurationOfPitch_cls"] = "不明"


In [19]:
print(all_df[:len(train_df)].groupby("DurationOfPitch").size())

DurationOfPitch
-60      121
 240       2
 300      18
 360      80
 420     239
 480     338
 540     327
 600     231
 660     160
 720     158
 780     203
 840     266
 900     284
 960     259
 1020    136
 1080     74
 1140     19
 1200     36
 1260     31
 1320     45
 1380     51
 1440     40
 1500     38
 1560     27
 1620     35
 1680     29
 1740     20
 1800     31
 1860     43
 1920     53
 1980     34
 2040     29
 2100     25
 2160      7
dtype: int64


In [20]:
####################
# "Occupation"の処理 #
####################
print(all_df.groupby(["Occupation"]).size())

Occupation
Large Business     721
Salaried          2801
Small Business    3456
dtype: int64


In [21]:
print(all_df[:len(train_df)].groupby("Occupation").size())

Occupation
Large Business     361
Salaried          1401
Small Business    1727
dtype: int64


In [22]:
##################
# "Genger"の処理 #
##################
# 先頭文字がMかmならMaleになる
for i in range(len(all_df)):

  # 先頭文字の取得(半角)
  head = unicodedata.normalize("NFKC", all_df.loc[i,"Gender"][:1]).lower()
  if(head == "f"):
    all_df.loc[i,"Gender"] = "Female"
  else:
    all_df.loc[i,"Gender"] = "Male"

# カテゴリ型にキャスト
all_df["Gender"] = all_df["Gender"].astype("object")
print(all_df.groupby(["Gender"]).size())

Gender
Female    2760
Male      4218
dtype: int64


In [23]:
print(all_df[:len(train_df)].groupby("Gender").size())

Gender
Female    1425
Male      2064
dtype: int64


In [24]:
##################################
# "NumberOfPersonVisiting"の処理 #
##################################
print(all_df.groupby(["NumberOfPersonVisiting"]).size())

NumberOfPersonVisiting
1.0     686
2.0    3163
3.0    2773
4.0     356
dtype: int64


In [25]:
print(all_df[:len(train_df)].groupby("NumberOfPersonVisiting").size())

NumberOfPersonVisiting
1.0     336
2.0    1608
3.0    1377
4.0     168
dtype: int64


In [26]:
##################################
# "NumberOfFollowups"の処理 #
##################################
# 欠損を-1で埋める
all_df["NumberOfFollowups"] = all_df["NumberOfFollowups"].fillna(-1)

# 100回以上は記録違いとして100で割る
all_df.loc[all_df["NumberOfFollowups"]>=100,"NumberOfFollowups"] = all_df.loc[all_df["NumberOfFollowups"]>=100,"NumberOfFollowups"] / 100

# 分布の確認
print(all_df.groupby(["NumberOfFollowups"]).size())

NumberOfFollowups
-1.0      57
 1.0     178
 2.0     213
 3.0    2561
 4.0    2777
 5.0    1122
 6.0      70
dtype: int64


In [27]:
# カテゴリ列の追加
all_df["NumberOfFollowups_cls"] = all_df["NumberOfFollowups"].copy()

all_df.loc[all_df["NumberOfFollowups"]==-1,"NumberOfFollowups_cls"] = "不明"
all_df.loc[all_df["NumberOfFollowups"]==1,"NumberOfFollowups_cls"] = "1回"
all_df.loc[all_df["NumberOfFollowups"]==2,"NumberOfFollowups_cls"] = "2回"
all_df.loc[all_df["NumberOfFollowups"]==3,"NumberOfFollowups_cls"] = "3回"
all_df.loc[all_df["NumberOfFollowups"]==4,"NumberOfFollowups_cls"] = "4回"
all_df.loc[all_df["NumberOfFollowups"]==5,"NumberOfFollowups_cls"] = "5回"
all_df.loc[all_df["NumberOfFollowups"]==6,"NumberOfFollowups_cls"] = "6回"

# 元のカラムを削除する
# all_df = all_df.drop("NumberOfFollowups", axis=1)

all_df["NumberOfFollowups_cls"] = all_df["NumberOfFollowups_cls"].astype("object")
print(all_df.groupby(["NumberOfFollowups_cls"]).size())

NumberOfFollowups_cls
1回     178
2回     213
3回    2561
4回    2777
5回    1122
6回      70
不明      57
dtype: int64


<ipython-input-27-90adc46ed6cf>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '不明' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_df.loc[all_df["NumberOfFollowups"]==-1,"NumberOfFollowups_cls"] = "不明"


In [28]:
print(all_df[:len(train_df)].groupby("NumberOfFollowups").size())

NumberOfFollowups
-1.0      33
 1.0      78
 2.0     108
 3.0    1296
 4.0    1411
 5.0     524
 6.0      39
dtype: int64


In [29]:
##########################
# "ProductPitched"の処理 #
##########################
# ギリシャ文字をアルファベットに変換
greek_string = "αβγδεζηθικλμνξοπρστυφχψωвꓢѕꭰտᗞı𐊡"
alphabet_string = "abgdezhqiklmnxoprstufcywbssdsdib"

def greek_to_alphabet(text):
    result = ""
    for char in text:
        if char in greek_string:
            result += alphabet_string[greek_string.index(char)]
        else:
            result += char  # ギリシャ文字以外の文字はそのまま
    return result

# Basic, Standard, Deluxe, Super Deluxe, kingに分ける
prefix = 0
others = []
# すべて小文字に変換し判定
for i in range(len(all_df)):
  prefix = unicodedata.normalize("NFKC", greek_to_alphabet(all_df.loc[i,"ProductPitched"][:2].lower())).lower()

  if(prefix == "ba"):
    all_df.loc[i,"ProductPitched"] = "Basic"

  elif(prefix == "st"):
    all_df.loc[i,"ProductPitched"] = "Standard"

  elif(prefix == "de"):
    all_df.loc[i,"ProductPitched"] = "Deluxe"

  elif(prefix == "su"):
    all_df.loc[i,"ProductPitched"] = "SuperDeluxe"

  elif(prefix == "ki"):
    all_df.loc[i,"ProductPitched"] = "King"

  else:
    others.append(i)

all_df["ProductPitched"] = all_df["ProductPitched"].astype("object")

# 分布の確認
print(all_df.groupby(["ProductPitched"]).size())

ProductPitched
Basic          2347
Deluxe         2091
King            286
Standard       1614
SuperDeluxe     640
dtype: int64


In [30]:
print(all_df[:len(train_df)].groupby("ProductPitched").size())

ProductPitched
Basic          1157
Deluxe         1045
King            126
Standard        841
SuperDeluxe     320
dtype: int64


In [31]:
#################################
# "PreferredPropertyStar"の処理 #
#################################
print(all_df.groupby(["PreferredPropertyStar"]).size())

PreferredPropertyStar
3.0    4328
4.0    2254
5.0     396
dtype: int64


In [32]:
print(all_df[:len(train_df)].groupby("PreferredPropertyStar").size())

PreferredPropertyStar
3.0    2147
4.0    1145
5.0     197
dtype: int64


In [33]:
#########################
# "NumberOfTrips"の処理 #
#########################
# nanは-1で補完
all_df["NumberOfTrips_num"] = [0]*len(all_df)
all_df["NumberOfTrips_num"] = -1

all_df["NumberOfTrips_num"] = all_df["NumberOfTrips_num"].astype(int)

all_df["NumberOfTrips"] = all_df["NumberOfTrips"].fillna("-1")

for i in range(len(all_df)):

  # 先頭文字の取得(半角)
  head = all_df.loc[i,"NumberOfTrips"][:1]

  if(head == "年"):
    all_df.loc[i,"NumberOfTrips_num"] = int(all_df.loc[i,"NumberOfTrips"][2:3])

  elif(head == "半"):
    all_df.loc[i,"NumberOfTrips_num"] = int(all_df.loc[i,"NumberOfTrips"][3:4])*2

  elif(head == "四"):
    all_df.loc[i,"NumberOfTrips_num"] = int(all_df.loc[i,"NumberOfTrips"][4:5])*4

  elif(head == "-1"):
    all_df.loc[i,"NumberOfTrips_num"] = -1

  else:
    all_df.loc[i,"NumberOfTrips_num"] = int(all_df.loc[i,"NumberOfTrips"])

print(all_df.groupby(["NumberOfTrips_num"]).size())

NumberOfTrips_num
-1      64
 1     801
 2    2277
 3    1706
 4     333
 5     977
 6     281
 7     513
 8      26
dtype: int64


In [34]:
# 欠損は最頻値で補完
# all_df.loc[all_df["NumberOfTrips_num"]<0,"NumberOfTrips_num"] = (all_df.loc[all_df["NumberOfTrips_num"]>0,["NumberOfTrips_num"]]).mode().values

all_df["NumberOfTrips"] = all_df["NumberOfTrips_num"].copy()
# all_df = all_df.drop("NumberOfTrips_num", axis=1)
print(all_df.groupby(["NumberOfTrips"]).size())

NumberOfTrips
-1      64
 1     801
 2    2277
 3    1706
 4     333
 5     977
 6     281
 7     513
 8      26
dtype: int64


In [35]:
print(all_df[:len(train_df)].groupby("NumberOfTrips").size())

NumberOfTrips
-1      22
 1     409
 2    1164
 3     820
 4     163
 5     515
 6     130
 7     256
 8      10
dtype: int64


In [36]:
#########################
# "Passport"の処理 #
#########################
print(all_df.groupby(["Passport"]).size())

Passport
0    6288
1     690
dtype: int64


In [37]:
print(all_df[:len(train_df)].groupby("Passport").size())

Passport
0    3131
1     358
dtype: int64


In [38]:
##################################
# "PitchSatisfactionScore"の処理 #
##################################
print(all_df.groupby(["PitchSatisfactionScore"]).size())

PitchSatisfactionScore
1    1439
2    2500
3    1344
4    1452
5     243
dtype: int64


In [39]:
print(all_df[:len(train_df)].groupby("PitchSatisfactionScore").size())

PitchSatisfactionScore
1     686
2    1269
3     692
4     739
5     103
dtype: int64


In [40]:
#########################
# "Designation"の処理 #
#########################

# ギリシャ文字をアルファベットに変換
greek_string = "αβγδεζηθικλμνξοπρστυφχψωвꓢѕꭰտᗞı𐊡Аае"
alphabet_string = "abgdezhqiklmnxoprstufcywbssdsdibaae"

def greek_to_alphabet(text):
    result = ""
    for char in text:
        if char in greek_string:
            result += alphabet_string[greek_string.index(char)]
        else:
            result += char  # ギリシャ文字以外の文字はそのまま
    return result

# 文字を変換する
prefix = 0
for i in range(len(all_df)):
  prefix = unicodedata.normalize("NFKC", greek_to_alphabet(all_df.loc[i,"Designation"][:1].lower())).lower()

  # Executive: 取締役、執行役員
  if(prefix == "e"):
    all_df.loc[i,"Designation"] = "Executive"

  # Vice President: 本部長
  elif(prefix == "v"):
    all_df.loc[i,"Designation"] = "VP"

  # Associate Vice President: 本部長補佐
  elif(prefix == "a"):
    all_df.loc[i,"Designation"] = "AVP"

  # Senior Manager: 次長
  elif(prefix == "s"):
    all_df.loc[i,"Designation"] = "Senior Manager"

  # Manager: 課長
  elif(prefix == "m"):
    all_df.loc[i,"Designation"] = "Manager"

all_df["Designation"] = all_df["Designation"].astype("object")

# 分布の確認
print(all_df.groupby(["Designation"]).size())

Designation
AVP                635
Executive         2330
Manager           2117
Senior Manager    1632
VP                 264
dtype: int64


In [41]:
print(all_df[:len(train_df)].groupby("Designation").size())

Designation
AVP                311
Executive         1150
Manager           1055
Senior Manager     854
VP                 119
dtype: int64


In [42]:
#########################
# "MonthlyIncome"の処理 #
#########################
import re

# 漢字の判定用
p = re.compile('[月]+')

# nanを-1で補完
all_df["MonthlyIncome"] = all_df["MonthlyIncome"].fillna("-1")

for i in range(len(all_df)):

  # 先頭文字の取得
  head = all_df.loc[i,"MonthlyIncome"][:1]

  # 先頭文字が漢字の場合
  if(p.search(head[0])!=None):
    tmp = all_df.loc[i,"MonthlyIncome"][2:-2]
    all_df.loc[i,"MonthlyIncome"] = float(tmp)*10000

# float型にキャストする
all_df["MonthlyIncome"] = all_df["MonthlyIncome"].astype(float)

print(all_df.groupby(["MonthlyIncome"]).size())

MonthlyIncome
-1.0         105
 200000.0      5
 238890.0      1
 239340.0      1
 240000.0      2
            ... 
 578460.0      1
 578490.0      1
 578670.0      1
 579015.0      1
 600000.0     10
Length: 4586, dtype: int64


In [43]:
all_df["MonthlyIncome_cls"] = all_df["MonthlyIncome"].copy()

all_df.loc[all_df["MonthlyIncome"]<=0,"MonthlyIncome_cls"] = "不明"
all_df.loc[(all_df["MonthlyIncome"]>0) & (all_df["MonthlyIncome"]<=250000),"MonthlyIncome_cls"] = "25万円以下"
all_df.loc[(all_df["MonthlyIncome"]>250000) & (all_df["MonthlyIncome"]<=300000),"MonthlyIncome_cls"] = "25〜30万円"
all_df.loc[(all_df["MonthlyIncome"]>300000) & (all_df["MonthlyIncome"]<=350000),"MonthlyIncome_cls"] = "30〜35万円"
all_df.loc[(all_df["MonthlyIncome"]>350000) & (all_df["MonthlyIncome"]<=400000),"MonthlyIncome_cls"] = "35〜40万円"
all_df.loc[(all_df["MonthlyIncome"]>400000), "MonthlyIncome_cls"] = "40万円以上"

# all_df = all_df.drop("MonthlyIncome", axis=1)

print(all_df.groupby(["MonthlyIncome_cls"]).size())

MonthlyIncome_cls
25〜30万円    1679
25万円以下       71
30〜35万円    2035
35〜40万円    1554
40万円以上     1534
不明          105
dtype: int64


<ipython-input-43-149bd5a7df1c>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '不明' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_df.loc[all_df["MonthlyIncome"]<=0,"MonthlyIncome_cls"] = "不明"


In [44]:
print(all_df[:len(train_df)].groupby("MonthlyIncome").size())

MonthlyIncome
-1.0         56
 200000.0     1
 238890.0     1
 240000.0     2
 241905.0     1
             ..
 575355.0     1
 576810.0     1
 578490.0     1
 578670.0     1
 600000.0     2
Length: 2542, dtype: int64


In [45]:
# ProdTaken
print(all_df.groupby(["ProdTaken"]).size())

ProdTaken
0.0    2992
1.0     497
dtype: int64


In [46]:
#########################
# "customer_info"の処理 #
#########################
import re

# 項目名
cont = []

# 区切り文字で分割する
for i in range(len(all_df)):
  tmp = re.split("[ 　,、\n\t/／\u3000]",all_df.loc[i,"customer_info"])
  tmp = list(filter(None, tmp))
  if(type(tmp)==list):
    for j in range(len(tmp)):
      if(tmp[j] not in cont):
        cont.append(tmp[j])

# キーワード抽出
c_car = [] # 車関係
c_child = [] # 子供関係
c_married = [] # 婚姻関係
c_other = [] # その他

for i in range(len(cont)):
  # 「車あり」、「車所持」、「自家用車あり」、「車保有」、「乗用車所持」、「自動車保有」
  # 「車なし」、「車未所持」、「自家用車なし」、「車保有なし」、「乗用車なし」、「自動車未所有」
  if((cont[i][0:1] == "車") or (cont[i][0:1] == "自") or (cont[i][0:1] == "乗")):
    c_car.append(cont[i])
  # 「こども1人」、「1児」、「子供1人」、「子供有り(1人)」、「子供有り」
  # 「こども2人」、「2児」、「子供2人」、「子供有り(2人)」
  # 「こども3人」、「3児」「子供3人」、「子供有り(3人)」
  # 「子供なし」、「子供無し」、「無子」、「子供ゼロ」、「非児童家庭」、「わからない」
  # 「子育て状況不明」、「子の数不詳」、「子供の数不明」
  elif((cont[i][0:1] == "子") or (cont[i][0:1] == "こ") or (cont[i][-1:] == "児") or (cont[i][-1:] == "子") or (cont[i][-1:] == "庭") or (cont[i][-1:] == "い")):
    c_child.append(cont[i])
  # 「結婚済み」、「離婚済み」、「未婚」、「独身」
  elif((cont[i][0:1] == "結") or (cont[i][0:1] == "離") or (cont[i][0:1] == "独") or (cont[i][0:1] == "離") or (cont[i][0:1] == "未")):
    c_married.append(cont[i])
  # 「1人」、「2人」、「3人」、「不明」
  else:
    c_other.append(cont[i])

In [47]:
# 新たなカラムを追加
all_df["CarHolder"] = "自動車未所有" # 車所有
all_df["NumofChild"] = "不明" # 子供の数
all_df["Married"] = "独身" # 婚姻歴

# 車
c_car_hold = ["車あり","車所持","自家用車あり","車保有","乗用車所持","自動車所有",
              "車未所持","自動車未所有","車保有なし","乗用車なし","自家用車なし","車なし"]

# 子供
c_child_x = ['子育て状況不明', '子の数不詳', '子供の数不明','わからない',"子供有り"]
c_child_0 = ['子供なし', '子供無し', '無子', '子供ゼロ', '非育児家庭']
c_child_1 = ["こども1人","1児","子供1人","子供有り(1人)"]
c_child_2 = ["こども2人","2児","子供2人","子供有り(2人)"]
c_child_3 = ["こども3人","3児","子供3人","子供有り(3人)"]

# 婚姻状況
c_pair = ["結婚済み"]
c_batu = ["離婚済み"]

# 区切り文字で分割する
for i in range(len(all_df)):
  tmp = re.split("[ 　,、\n\t/／\u3000]",all_df.loc[i,"customer_info"])
  tmp = list(filter(None, tmp))
  if(type(tmp)==list):
    for j in range(len(tmp)):
      # 車の情報
      if(tmp[j] in c_car_hold):
        if(tmp[j] in ["車あり","車所持","自家用車あり","車保有","乗用車所持","自動車所有"]):
          all_df.loc[i, "CarHolder"] = "自動車所有"
      # 子供の情報
      if(tmp[j] in c_child_x):
        all_df.loc[i, "NumofChild"] = "不明"
      if(tmp[j] in c_child_0):
        all_df.loc[i, "NumofChild"] = "子供なし"
      if(tmp[j] in c_child_1):
        all_df.loc[i, "NumofChild"] = "子供1人"
      if(tmp[j] in c_child_2):
        all_df.loc[i, "NumofChild"] = "子供2人"
      if(tmp[j] in c_child_3):
        all_df.loc[i, "NumofChild"] = "子供3人"
      # 婚姻歴
      if(tmp[j] in c_pair):
        all_df.loc[i, "Married"] = "結婚済み"
      if(tmp[j] in c_batu):
        all_df.loc[i, "Married"] = "離婚済み"

# 使用済みカラム抹消
all_df = all_df.drop("customer_info", axis=1)

In [48]:
# 車保有率
all_df["CarHolder"] = all_df["CarHolder"].astype("object")
print(all_df.groupby(["CarHolder"]).size())

CarHolder
自動車所有     3359
自動車未所有    3619
dtype: int64


In [49]:
print(all_df[:len(train_df)].groupby("CarHolder").size())

CarHolder
自動車所有     1704
自動車未所有    1785
dtype: int64


In [50]:
# 子供の数
all_df["NumofChild"] = all_df["NumofChild"].astype("object")
print(all_df.groupby(["NumofChild"]).size())

NumofChild
不明       395
子供1人    2945
子供2人    1600
子供3人     129
子供なし    1909
dtype: int64


In [51]:
print(all_df[:len(train_df)].groupby("NumofChild").size())

NumofChild
不明       197
子供1人    1476
子供2人     763
子供3人      68
子供なし     985
dtype: int64


In [52]:
# 婚姻歴
all_df["Married"] = all_df["Married"].astype("object")
print(all_df.groupby(["Married"]).size())

Married
独身      2444
結婚済み    2846
離婚済み    1688
dtype: int64


In [53]:
print(all_df[:len(train_df)].groupby("Married").size())

Married
独身      1227
結婚済み    1384
離婚済み     878
dtype: int64


In [54]:
all_df.columns

Index(['id', 'Age', 'TypeofContact', 'CityTier', 'DurationOfPitch',
       'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'ProductPitched', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'Designation', 'MonthlyIncome', 'ProdTaken',
       'Age_cls', 'DurationOfPitch_cls', 'NumberOfFollowups_cls',
       'NumberOfTrips_num', 'MonthlyIncome_cls', 'CarHolder', 'NumofChild',
       'Married'],
      dtype='object')

In [55]:
print(all_df.dtypes)

id                          int64
Age                         int64
TypeofContact              object
CityTier                    int64
DurationOfPitch             int64
Occupation                 object
Gender                     object
NumberOfPersonVisiting    float64
NumberOfFollowups         float64
ProductPitched             object
PreferredPropertyStar     float64
NumberOfTrips               int64
Passport                    int64
PitchSatisfactionScore      int64
Designation                object
MonthlyIncome             float64
ProdTaken                 float64
Age_cls                    object
DurationOfPitch_cls        object
NumberOfFollowups_cls      object
NumberOfTrips_num           int64
MonthlyIncome_cls          object
CarHolder                  object
NumofChild                 object
Married                    object
dtype: object


In [ ]:
# カテゴリ化した数値カラム削除
all_df = all_df.drop(["Age", "DurationOfPitch", "NumberOfTrips_num", "NumberOfFollowups", "MonthlyIncome"],axis=1)

In [ ]:
# 順番を整列させる
col = ['id', 'TypeofContact', 'CityTier',
       'Occupation', 'Gender', 'NumberOfPersonVisiting',
       'ProductPitched', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'Designation', 'ProdTaken',
       'Married', 'CarHolder', 'NumofChild', 'Age_cls', 'DurationOfPitch_cls',
       'NumberOfFollowups_cls', 'MonthlyIncome_cls',
       ]

all_df = all_df.reindex(columns=col)

In [ ]:
train_df

,id,TypeofContact,CityTier,Occupation,Gender,NumberOfPersonVisiting,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,ProdTaken,customer_marriage,customer_car,customer_child,age_cls,DurationOfPitch_cls,NumberOfFollowups_cls,MonthlyIncome_cls
0,0,Self Enquiry,2,Large Business,Male,1.0,basic,3.0,5.0,1,4,executive,1,未婚,自動車未所有,子供なし,46~50歳,801~900s,4回,25~30万円
1,1,Company Invited,1,Salaried,Male,1.0,standard,3.0,2.0,1,4,senior manager,0,離婚済み,自動車所有,子供なし,56歳以上,801~900s,4回,40万円以上
2,2,Self Enquiry,1,Large Business,Female,1.0,basic,3.0,4.0,0,4,executive,1,結婚済み,自動車未所有,子供なし,不明,501~600s,3回,25~30万円
3,3,Self Enquiry,2,Small Business,Female,1.0,standard,4.0,1.0,0,5,senior manager,0,離婚済み,自動車所有,子供なし,36~40歳,1001~1100s,3回,30~35万円
4,4,Company Invited,3,Small Business,Female,1.0,basic,4.0,4.0,0,4,executive,1,独身,自動車所有,子供なし,46~50歳,1001~1100s,3回,25~30万円
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3484,3484,Self Enquiry,2,Salaried,Male,2.0,basic,3.0,3.0,0,1,executive,1,離婚済み,自動車所有,子供なし,36~40歳,1501s以上,3回,25~30万円
3485,3485,Self Enquiry,1,Large Business,Male,3.0,basic,5.0,5.0,0,3,executive,0,結婚済み,自動車所有,子供2人,41~45歳,501~600s,3回,25~30万円
3486,3486,Self Enquiry,1,Small Business,Female,3.0,standard,3.0,5.0,0,4,senior manager,0,独身,自動車所有,子供1人,31~35歳,801~900s,2回,30~35万円
3487,3487,Company Invited,2,Salaried,Male,3.0,king,3.0,7.0,1,4,vp,1,結婚済み,自動車未所有,子供2人,56歳以上,801~900s,6回,40万円以上


In [ ]:
test_df

,id,TypeofContact,CityTier,Occupation,Gender,NumberOfPersonVisiting,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,customer_marriage,customer_car,customer_child,age_cls,DurationOfPitch_cls,NumberOfFollowups_cls,MonthlyIncome_cls
0,3489,Self Enquiry,2,Small Business,Male,1.0,super deluxe,3.0,7.0,0,3,avp,結婚済み,自動車所有,子供なし,46~50歳,701~800s,4回,40万円以上
1,3490,Self Enquiry,2,Small Business,Female,1.0,standard,3.0,4.0,1,3,senior manager,結婚済み,自動車未所有,子供なし,31~35歳,701~800s,4回,25~30万円
2,3491,Self Enquiry,1,Salaried,Female,1.0,basic,3.0,1.0,0,3,executive,離婚済み,自動車未所有,子供なし,21~25歳,501~600s,4回,25~30万円
3,3492,Company Invited,2,Salaried,Male,1.0,basic,4.0,1.0,0,3,senior manager,離婚済み,自動車所有,子供なし,21~25歳,401~500s,4回,25~30万円
4,3493,Company Invited,1,Salaried,Male,1.0,basic,3.0,1.0,0,4,executive,独身,自動車所有,子供なし,41~45歳,401~500s,4回,25~30万円
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3484,6973,Company Invited,1,Small Business,Female,1.0,basic,3.0,2.0,0,4,executive,結婚済み,自動車未所有,子供なし,41~45歳,801~900s,3回,25~30万円
3485,6974,Company Invited,1,Salaried,Male,3.0,deluxe,3.0,3.0,0,3,manager,結婚済み,自動車所有,子供1人,41~45歳,1501s以上,5回,30~35万円
3486,6975,Self Enquiry,2,Small Business,Male,2.0,basic,3.0,2.0,0,3,executive,独身,自動車未所有,子供なし,21~25歳,1201~1300s,3回,25~30万円
3487,6976,Self Enquiry,1,Small Business,Male,2.0,basic,3.0,2.0,0,3,executive,結婚済み,自動車未所有,子供1人,21~25歳,501~600s,3回,25~30万円


In [ ]:
train_df.isnull().sum()

,0
id,0
TypeofContact,0
CityTier,0
Occupation,0
Gender,0
NumberOfPersonVisiting,0
ProductPitched,0
PreferredPropertyStar,0
NumberOfTrips,0
Passport,0


In [ ]:
test_df.isnull().sum()

,0
id,0
TypeofContact,0
CityTier,0
Occupation,0
Gender,0
NumberOfPersonVisiting,0
ProductPitched,0
PreferredPropertyStar,0
NumberOfTrips,0
Passport,0


In [56]:
y = train_df["ProdTaken"]
X = train_df.drop(["id", "ProdTaken"], axis=1)

In [ ]:
X.describe()

,CityTier,NumberOfPersonVisiting,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore
count,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000
mean,1.697048,2.394669,3.441101,3.187160,0.102608,2.513901
std,0.671500,0.726612,0.599633,1.729963,0.303490,1.115412
min,1.000000,1.000000,3.000000,1.000000,0.000000,1.000000
25%,1.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,2.000000,2.000000,3.000000,3.000000,0.000000,2.000000
75%,2.000000,3.000000,4.000000,5.000000,0.000000,3.000000
max,3.000000,4.000000,5.000000,8.000000,1.000000,5.000000


In [57]:
# 結合を戻す
train = all_df.iloc[:len(train_df)].copy()
test = all_df.iloc[len(train_df):].reset_index(drop=True).copy()

In [60]:
test

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,MonthlyIncome,ProdTaken,Age_cls,DurationOfPitch_cls,NumberOfFollowups_cls,NumberOfTrips_num,MonthlyIncome_cls,CarHolder,NumofChild,Married
0,3489,48,Self Enquiry,2,780,Small Business,Male,1.0,4.0,SuperDeluxe,...,496950.0,NaN,46〜50歳,701〜800s,4回,7,40万円以上,自動車所有,子供なし,結婚済み
1,3490,35,Self Enquiry,2,720,Small Business,Female,1.0,4.0,Standard,...,300000.0,NaN,31〜35歳,701〜800s,4回,4,25〜30万円,自動車未所有,子供なし,結婚済み
2,3491,25,Self Enquiry,1,540,Salaried,Female,1.0,4.0,Basic,...,260000.0,NaN,21〜25歳,501〜600s,4回,1,25〜30万円,自動車未所有,子供なし,離婚済み
3,3492,21,Company Invited,2,420,Salaried,Male,1.0,4.0,Basic,...,259875.0,NaN,21〜25歳,401〜500s,4回,1,25〜30万円,自動車所有,子供なし,離婚済み
4,3493,41,Company Invited,1,420,Salaried,Male,1.0,4.0,Basic,...,268830.0,NaN,41〜45歳,401〜500s,4回,1,25〜30万円,自動車所有,子供なし,独身
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3484,6973,41,Company Invited,1,840,Small Business,Female,1.0,3.0,Basic,...,261840.0,NaN,41〜45歳,801〜900s,3回,2,25〜30万円,自動車未所有,子供なし,結婚済み
3485,6974,44,Company Invited,1,2100,Salaried,Male,3.0,5.0,Deluxe,...,349770.0,NaN,41〜45歳,1501s以上,5回,3,30〜35万円,自動車所有,子供1人,結婚済み
3486,6975,24,Self Enquiry,2,1260,Small Business,Male,2.0,3.0,Basic,...,270000.0,NaN,21〜25歳,1201〜1300s,3回,2,25〜30万円,自動車未所有,子供なし,独身
3487,6976,25,Self Enquiry,1,540,Small Business,Male,2.0,3.0,Basic,...,272430.0,NaN,21〜25歳,501〜600s,3回,2,25〜30万円,自動車未所有,子供1人,結婚済み


In [78]:
test.groupby("MonthlyIncome").size()

,0
MonthlyIncome,
-1.0,49
200000.0,4
239340.0,1
240735.0,1
242655.0,1
...,...
577065.0,1
577380.0,1
578460.0,1


In [ ]:
# 欠損値補完
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      3489 non-null   int64  
 1   TypeofContact           3483 non-null   object 
 2   CityTier                3489 non-null   int64  
 3   Occupation              3489 non-null   object 
 4   Gender                  3489 non-null   object 
 5   NumberOfPersonVisiting  3489 non-null   float64
 6   ProductPitched          3489 non-null   object 
 7   PreferredPropertyStar   3489 non-null   float64
 8   NumberOfTrips           3489 non-null   int64  
 9   Passport                3489 non-null   int64  
 10  PitchSatisfactionScore  3489 non-null   int64  
 11  Designation             3489 non-null   object 
 12  ProdTaken               3489 non-null   float64
 13  Married                 3489 non-null   object 
 14  CarHolder               3489 non-null   

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      3489 non-null   int64  
 1   TypeofContact           3477 non-null   object 
 2   CityTier                3489 non-null   int64  
 3   Occupation              3489 non-null   object 
 4   Gender                  3489 non-null   object 
 5   NumberOfPersonVisiting  3489 non-null   float64
 6   ProductPitched          3489 non-null   object 
 7   PreferredPropertyStar   3489 non-null   float64
 8   NumberOfTrips           3489 non-null   int64  
 9   Passport                3489 non-null   int64  
 10  PitchSatisfactionScore  3489 non-null   int64  
 11  Designation             3489 non-null   object 
 12  ProdTaken               0 non-null      float64
 13  Married                 3489 non-null   object 
 14  CarHolder               3489 non-null   

In [ ]:
train.fillna(train.mode().iloc[0], inplace=True)
test.fillna(test.mode().iloc[0], inplace=True)

In [ ]:
train.loc[train["NumberOfTrips"]==-1,"NumberOfTrips"] = train.loc[train["NumberOfTrips"]>0,"NumberOfTrips"].median()
test.loc[test["NumberOfTrips"]==-1,"NumberOfTrips"] = test.loc[test["NumberOfTrips"]>0,"NumberOfTrips"].median()

In [ ]:
train.isnull().sum()

,0
id,0
TypeofContact,0
CityTier,0
Occupation,0
Gender,0
NumberOfPersonVisiting,0
ProductPitched,0
PreferredPropertyStar,0
NumberOfTrips,0
Passport,0


In [ ]:
test = test.drop("ProdTaken",axis=1)

In [ ]:
test.isnull().sum()

,0
id,0
TypeofContact,0
CityTier,0
Occupation,0
Gender,0
NumberOfPersonVisiting,0
ProductPitched,0
PreferredPropertyStar,0
NumberOfTrips,0
Passport,0


In [ ]:
y = train["ProdTaken"]
X = train.drop(["id", "ProdTaken"], axis=1)

In [ ]:
X.describe()

,CityTier,NumberOfPersonVisiting,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore
count,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000
mean,1.697048,2.394669,3.441101,3.187160,0.102608,2.513901
std,0.671500,0.726612,0.599633,1.729963,0.303490,1.115412
min,1.000000,1.000000,3.000000,1.000000,0.000000,1.000000
25%,1.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,2.000000,2.000000,3.000000,3.000000,0.000000,2.000000
75%,2.000000,3.000000,4.000000,5.000000,0.000000,3.000000
max,3.000000,4.000000,5.000000,8.000000,1.000000,5.000000


In [ ]:
test.describe()

,id,CityTier,NumberOfPersonVisiting,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore
count,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000
mean,5233.000000,1.682717,2.407567,3.431929,3.205216,0.095156,2.500143
std,1007.331872,0.671710,0.741628,0.599618,1.732139,0.293473,1.153904
min,3489.000000,1.000000,1.000000,3.000000,1.000000,0.000000,1.000000
25%,4361.000000,1.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,5233.000000,2.000000,2.000000,3.000000,3.000000,0.000000,2.000000
75%,6105.000000,2.000000,3.000000,4.000000,5.000000,0.000000,3.000000
max,6977.000000,3.000000,4.000000,5.000000,8.000000,1.000000,5.000000


In [ ]:
for feature in train.columns:
    print(train.groupby(feature).size())
    print("")

id
0       1
1       1
2       1
3       1
4       1
       ..
3484    1
3485    1
3486    1
3487    1
3488    1
Length: 3489, dtype: int64

TypeofContact
Company Invited    1289
Self Enquiry       2200
dtype: int64

CityTier
1    1475
2    1596
3     418
dtype: int64

Occupation
Large Business     361
Salaried          1401
Small Business    1727
dtype: int64

Gender
Female    1425
Male      2064
dtype: int64

NumberOfPersonVisiting
1.0     336
2.0    1608
3.0    1377
4.0     168
dtype: int64

ProductPitched
basic           1157
deluxe          1045
king             126
standard         841
super deluxe     320
dtype: int64

PreferredPropertyStar
3.0    2147
4.0    1145
5.0     197
dtype: int64

NumberOfTrips
1.0     409
2.0    1186
3.0     820
4.0     163
5.0     515
6.0     130
7.0     256
8.0      10
dtype: int64

Passport
0    3131
1     358
dtype: int64

PitchSatisfactionScore
1     686
2    1269
3     692
4     739
5     103
dtype: int64

Designation
avp                311
execu

In [ ]:
train.describe()

,id,CityTier,NumberOfPersonVisiting,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,ProdTaken
count,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000
mean,1744.000000,1.697048,2.394669,3.441101,3.187160,0.102608,2.513901,0.142448
std,1007.331872,0.671500,0.726612,0.599633,1.729963,0.303490,1.115412,0.349559
min,0.000000,1.000000,1.000000,3.000000,1.000000,0.000000,1.000000,0.000000
25%,872.000000,1.000000,2.000000,3.000000,2.000000,0.000000,2.000000,0.000000
50%,1744.000000,2.000000,2.000000,3.000000,3.000000,0.000000,2.000000,0.000000
75%,2616.000000,2.000000,3.000000,4.000000,5.000000,0.000000,3.000000,0.000000
max,3488.000000,3.000000,4.000000,5.000000,8.000000,1.000000,5.000000,1.000000


In [ ]:
test.describe()

,id,CityTier,NumberOfPersonVisiting,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore
count,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000,3489.000000
mean,5233.000000,1.682717,2.407567,3.431929,3.205216,0.095156,2.500143
std,1007.331872,0.671710,0.741628,0.599618,1.732139,0.293473,1.153904
min,3489.000000,1.000000,1.000000,3.000000,1.000000,0.000000,1.000000
25%,4361.000000,1.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,5233.000000,2.000000,2.000000,3.000000,3.000000,0.000000,2.000000
75%,6105.000000,2.000000,3.000000,4.000000,5.000000,0.000000,3.000000
max,6977.000000,3.000000,4.000000,5.000000,8.000000,1.000000,5.000000


# 特徴量エンジニアリング

# 学習準備

# ここから学習作業

In [ ]:
# オブジェクト型を抽出する
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

In [ ]:
categorical_features

['TypeofContact',
 'Occupation',
 'Gender',
 'ProductPitched',
 'Designation',
 'customer_marriage',
 'customer_car',
 'customer_child',
 'age_cls',
 'DurationOfPitch_cls',
 'NumberOfFollowups_cls',
 'MonthlyIncome_cls']

In [ ]:
import pandas as pd
import numpy as np
import optuna
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.model_selection import train_test_split, StratifiedKFold
import sklearn.metrics

In [ ]:
# Optunaを使用する
import optuna
import lightgbm as lgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import KFold # K分割
import lightgbm as lgb
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold

def objective(trial):
    # パラメータの指定
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 300),
        'depth' : trial.suggest_int('depth', 4, 10),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50),
        'eval_metric' :'AUC',
    }

    # KFold
    cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=32)
    auc_score = []
    pred_score = []
    pred_test_df = test_df.drop(["id"], axis=1)

    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), start=0):
        X_train = X.iloc[train_idx, :]
        y_train = y[train_idx]
        X_val = X.iloc[val_idx, :]
        y_val = y[val_idx]

        cb_model = CatBoostClassifier(**params)

        cb_model.fit(
             X_train, y_train,
             cat_features=categorical_features,
             eval_set=(X_val, y_val),
             use_best_model=True,
             verbose=False,
            )

        # AUC算出
        auc_score.append(cb_model.get_best_score()["validation"]["AUC"])
        # # testデータのスコア算出
        # pred_score.append(cb_model.predict_proba(pred_test_df)[:,1])

    return sum(auc_score)/len(auc_score)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-05-31 14:32:25,714] A new study created in memory with name: no-name-e76fd2d4-5bbb-4fe6-a0c2-a6656e152c95
<ipython-input-175-1f8b0b81eb75>:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-175-1f8b0b81eb75>:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
[I 2025-05-31 14:32:26,665] Trial 0 finished with value: 0.8172699331536065 and parameters: {'iterations': 68, 'depth': 10, 'learning_rate': 0.025804636741835557, 'random_strength': 32, 'bagging_temperature': 38.72731044631443, 'od_type': '

In [ ]:
print(study.best_trial)
print(study.best_params)

FrozenTrial(number=89, state=1, values=[0.8437166367506183], datetime_start=datetime.datetime(2025, 5, 31, 14, 38, 45, 471027), datetime_complete=datetime.datetime(2025, 5, 31, 14, 38, 47, 242522), params={'iterations': 192, 'depth': 5, 'learning_rate': 0.2456034803864835, 'random_strength': 53, 'bagging_temperature': 23.684038757708525, 'od_type': 'Iter', 'od_wait': 38}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'iterations': IntDistribution(high=300, log=False, low=50, step=1), 'depth': IntDistribution(high=10, log=False, low=4, step=1), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None), 'random_strength': IntDistribution(high=100, log=False, low=0, step=1), 'bagging_temperature': FloatDistribution(high=100.0, log=True, low=0.01, step=None), 'od_type': CategoricalDistribution(choices=('IncToDec', 'Iter')), 'od_wait': IntDistribution(high=50, log=False, low=10, step=1)}, trial_id=89, value=None)
{'iterations': 192, 'depth': 5, 'le

In [ ]:
for i, trial in enumerate(study.best_trials):
  print(trial.params)
  print([j for j in trial.values])

{'iterations': 192, 'depth': 5, 'learning_rate': 0.2456034803864835, 'random_strength': 53, 'bagging_temperature': 23.684038757708525, 'od_type': 'Iter', 'od_wait': 38}
[0.8437166367506183]


In [ ]:
best_params = study.best_params

In [ ]:
best_params['eval_metric'] = 'AUC'

In [ ]:
best_params

{'iterations': 192,
 'depth': 5,
 'learning_rate': 0.2456034803864835,
 'random_strength': 53,
 'bagging_temperature': 23.684038757708525,
 'od_type': 'Iter',
 'od_wait': 38,
 'eval_metric': 'AUC'}

In [ ]:
best_params={'iterations': 171,
 'depth': 6,
 'learning_rate': 0.29397858515878483,
 'random_strength': 68,
 'bagging_temperature': 1.809748091233205,
 'od_type': 'IncToDec',
 'od_wait': 30,
 'eval_metric': 'AUC'}

In [ ]:
train_pred_df = pd.DataFrame()
cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=32)
auc_score = []
pred_score = []
pred_test = test.drop(["id"], axis=1)

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), start=0):
    X_train = X.iloc[train_idx, :]
    y_train = y[train_idx]
    X_val = X.iloc[val_idx, :]
    y_val = y[val_idx]

    cb_model = CatBoostClassifier(**best_params)

    cb_model.fit(
         X_train, y_train,
         cat_features=categorical_features,
         eval_set=(X_val, y_val),
         use_best_model=True,
         verbose=True,
        )

    # AUC算出
    auc_score.append(cb_model.get_best_score()["validation"]["AUC"])
    # testデータのスコア算出
    pred_score.append(cb_model.predict_proba(pred_test)[:,1])
    # trainデータのスコア算出
    pred_valid = cb_model.predict_proba(X_val)[:,1]
    tmp_df = pd.DataFrame(
        {
            "catboost_Score": pred_valid,
            "true": y_val,
        },
        index=X_val.index,
    )
    train_pred_df = pd.concat([train_pred_df, tmp_df], axis=0)
train_pred_df = train_pred_df.sort_index()

0:	test: 0.5999901	best: 0.5999901 (0)	total: 1.77ms	remaining: 302ms
1:	test: 0.5999901	best: 0.5999901 (0)	total: 2.86ms	remaining: 242ms
2:	test: 0.6179742	best: 0.6179742 (2)	total: 3.83ms	remaining: 214ms
3:	test: 0.6193563	best: 0.6193563 (3)	total: 5.09ms	remaining: 213ms
4:	test: 0.6193563	best: 0.6193563 (3)	total: 6.27ms	remaining: 208ms
5:	test: 0.6193563	best: 0.6193563 (3)	total: 7.39ms	remaining: 203ms
6:	test: 0.6193563	best: 0.6193563 (3)	total: 8.29ms	remaining: 194ms
7:	test: 0.7816737	best: 0.7816737 (7)	total: 9.47ms	remaining: 193ms
8:	test: 0.7816737	best: 0.7816737 (7)	total: 10.2ms	remaining: 183ms
9:	test: 0.7816737	best: 0.7816737 (7)	total: 10.8ms	remaining: 175ms
10:	test: 0.7816737	best: 0.7816737 (7)	total: 11.6ms	remaining: 168ms
11:	test: 0.7735948	best: 0.7816737 (7)	total: 12.7ms	remaining: 169ms
12:	test: 0.7712255	best: 0.7816737 (7)	total: 13.9ms	remaining: 168ms
13:	test: 0.7620113	best: 0.7816737 (7)	total: 15ms	remaining: 168ms
14:	test: 0.786149

In [ ]:
auc_score

[0.8587929445833882,
 0.8544820323812031,
 0.8240752928787679,
 0.8407494145199064,
 0.849622324108586,
 0.8452023617112512,
 0.8371540719728205]

In [ ]:
sum(auc_score)/len(auc_score)

0.844296920307989

In [ ]:
average_pred = np.mean(pred_score, axis=0)

In [ ]:
print(pred_score)
print(average_pred)

[array([0.10459695, 0.17743113, 0.27470821, ..., 0.683403  , 0.24457109,
       0.01461685]), array([0.08796416, 0.14000424, 0.28080759, ..., 0.72818812, 0.30797898,
       0.00443203]), array([0.09913188, 0.20622362, 0.28943189, ..., 0.57686552, 0.27126574,
       0.0024867 ]), array([0.05253394, 0.52175776, 0.20843719, ..., 0.72949162, 0.33342381,
       0.00526467]), array([0.07860216, 0.2478917 , 0.23527592, ..., 0.64622942, 0.23815092,
       0.02001418]), array([0.10784412, 0.31400049, 0.2084629 , ..., 0.73046858, 0.23289193,
       0.02230413]), array([0.10814342, 0.25016057, 0.1453782 , ..., 0.77598913, 0.24995842,
       0.016364  ])]
[0.09125952 0.26535279 0.23464313 ... 0.69580506 0.26832013 0.0122118 ]


In [ ]:
submit_df = pd.read_csv("/content/sample_submit.csv", header=None)
submit_df.head()
submit_df[1] = average_pred
submit_df.to_csv("submit.csv", index=False, header=None)

# 重要度を比較する

In [ ]:
# 重要度の算出
feature_importances = 0
for i in range(5):
  feature_importances += pd.DataFrame(models[i].get_feature_importance(),columns=['importance'])

feature=pd.DataFrame(test.columns,columns=['feature'])
o=pd.concat([feature,feature_importances],axis=1)
o = o.sort_values("importance", ascending=False)
print(o)

In [ ]:
o

In [ ]:
plt.figure(figsize=(16, 16))
sns.barplot(data=o.sort_values(by='importance', ascending=False), x='importance', y='feature')
plt.title('Feature Importances ')

In [ ]:
import shap
import lightgbm as lgb
import matplotlib.pyplot as plt
shap.initjs()

# 各FOLD、SEEDの和 (相対値なのでこれで良しとする)
shapval = 0
for i in range(5):
  explainer = shap.TreeExplainer(models[i])
  shap_values = explainer.shap_values(train.drop("ProdTaken",axis=1))
  shapval = shapval + shap_values

shap.summary_plot(shapval, train.drop("ProdTaken",axis=1),plot_type="bar")